# Neural Network for Classification

---

In [6]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm

In [7]:
test_data = pd.read_csv('bank-note/test.csv', header=None)
train_data = pd.read_csv('bank-note/train.csv', header=None)

In [8]:
# first 7 columns are features, last column (Slump) is output
columns = ['var', 'skew', 'curt', 'ent', 'label']
features = columns[:-1]
output = columns[-1]

test_data.columns = columns
train_data.columns = columns

In [9]:
train_X = train_data.iloc[:,:-1].values
test_X = test_data.iloc[:,:-1].values

In [10]:
train_X = np.hstack((train_X, np.ones(train_X.shape[0]).reshape(-1,1)))
test_X = np.hstack((test_X, np.ones(test_X.shape[0]).reshape(-1,1)))

In [11]:
train_y = train_data.iloc[:,-1].values
test_y = test_data.iloc[:,-1].values

In [12]:
# Convert labels to {-1,1}
train_y = np.array([1 if x else -1 for x in train_y])
test_y = np.array([1 if x else -1 for x in test_y])

# reshape to 2D array
train_y = train_y.reshape(-1,1)
test_y = test_y.reshape(-1,1)

In [13]:
train_data.head()

,var,skew,curt,ent,label
0,3.848100,10.15390,-3.85610,-4.22280,0
1,4.004700,0.45937,1.36210,1.61810,0
2,-0.048008,-1.60370,8.47560,0.75558,0
3,-1.266700,2.81830,-2.42600,-1.88620,1
4,2.203400,5.99470,0.53009,0.84998,0


In [10]:
train_X[:5]

array([[ 3.8481  , 10.1539  , -3.8561  , -4.2228  ,  1.      ],
       [ 4.0047  ,  0.45937 ,  1.3621  ,  1.6181  ,  1.      ],
       [-0.048008, -1.6037  ,  8.4756  ,  0.75558 ,  1.      ],
       [-1.2667  ,  2.8183  , -2.426   , -1.8862  ,  1.      ],
       [ 2.2034  ,  5.9947  ,  0.53009 ,  0.84998 ,  1.      ]])

In [9]:
train_y[:5]

array([[-1],
       [-1],
       [-1],
       [ 1],
       [-1]])

In [17]:
TEST = True

In [14]:
def truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [127]:
class NeuralNet:
    def __init__(self, n_input_nodes, n_output_nodes, n_hidden_nodes):
        #self.X = X
        #self.y = y
        self.n_input_nodes = n_input_nodes
        self.n_output_nodes = n_output_nodes
        self.n_hidden_nodes = n_hidden_nodes
        
        self.weights = []
        self.biases = []
        
        self.init_weights()
        
    def init_weights(self):
        # weights from input to hidden layer #1
        rad = 1 / np.sqrt(self.n_input_nodes)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        weights1 = X.rvs((self.n_hidden_nodes-1, self.n_input_nodes ))
        #bias1 = X.rvs((self.n_hidden_nodes-1, 1))
        self.weights.append(weights1)
        #self.biases.append(bias1)
        
        # weights from hidden layer #1 to #2
        rad = 1 / np.sqrt(self.n_hidden_nodes)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        weights2 = X.rvs((self.n_hidden_nodes-1, self.n_hidden_nodes))
        #bias2 = X.rvs((self.n_hidden_nodes-1, 1))
        self.weights.append(weights2)
        #self.biases.append(bias2)
        
        # weights from hidden layer #2 to output
        rad = 1 / np.sqrt(self.n_hidden_nodes)
        X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
        weights3 = X.rvs((self.n_output_nodes, self.n_hidden_nodes))
        #bias3 = X.rvs((1))
        self.weights.append(weights3)
        #self.biases.append(bias3)
        
        if TEST:
            self.weights = []
            #self.biases = []
            weights1 = np.array([[-1,1],[-2,2],[-3,3]]).T
            #bias1 = np.array([-1,1])
            weights2 = np.array([[-1,1],[-2,2],[-3,3]]).T
            #bias2 = np.array([-1,1])
            weights3 = np.array([[-1],[2],[-1.5]]).T
            #bias3 = np.array([-1])
            
            self.weights.append(weights1)
            self.weights.append(weights2)
            self.weights.append(weights3)
            #self.biases.append(bias1)
            #self.biases.append(bias2)
            #self.biases.append(bias3)
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_deriv(self, x):
        return self.sigmoid(x)*(1 - self.sigmoid(x))
        
    def feedforward(self, x):
        self.z1 = self.weights[0].dot(x)
        self.layer1 = np.insert(self.sigmoid(self.z1),0,1)
        print('layer1 z={} a={}'.format(self.z1,self.layer1))
        self.z2 = self.weights[1].dot(self.layer1)
        self.layer2 = np.insert(self.sigmoid(self.z2),0,1)
        print('layer2 z={} a={}'.format(self.z2,self.layer2))
        self.z_out = self.weights[2].dot(self.layer2)
        self.a_out = self.sigmoid(self.z_out)
        
        return self.z_out
    
    def backprop(self,y):
        
        delta_o = (y - self.a_out)*self.sigmoid_deriv(self.z_out)
        print(self.weights[2][:,1:].T.dot(delta_o))
        delta2 = self.weights[2][:,1:].T.dot(delta_o)*self.sigmoid_deriv(self.z2)
        delta1 = self.weights[1][:,1:].T.dot(delta2)*self.sigmoid_deriv(self.z1)
        
        dw_o = delta_o.dot(self.a_out.T)
        dw_2 = delta2.dot(self.layer2.T)
        dw_1 = delta1.dot(self.layer1.T)
        
        dw = [dw_o, dw_2, dw_1]
        
        return dw
        
        
        #a = (self.output - 1) * self.sigmoid_deriv(self.output)
        #self.d_weights3 = np.dot(self.layer2.T, (a))
        #self.d_weights2 = np.dot(self.layer1.T,  (np.dot(a, self.weights3.T) * self.sigmoid_deriv(self.layer2)))
    
    def train(self, X, y):
        pass
    
    def predict(self):
        pass
    


In [50]:
for i in reversed(range(2)):
    print(i)

1
0


In [40]:
n_input_nodes = 2
n_output_nodes = 1
n_hidden_nodes = 2

In [38]:
rad = 1 / np.sqrt(n_input_nodes)
X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
weights1 = X.rvs((n_input_nodes+1, n_hidden_nodes))

weights1

array([[ 0.08913063, -0.08668771],
       [-0.09828425, -0.69009948],
       [-0.13346366, -0.48029352]])

In [ ]:
rad = 1 / np.sqrt(n_hidden_nodes)
X = truncated_normal(mean=0, sd=1, low=-rad, upp=rad)
weights_hidden_out = X.rvs((n_output_nodes, n_hidden_nodes))

In [ ]:
d_weights2 = np.dot(self.layer1.T, ((self.y - self.output) * sigmoid_derivative(self.output)))
d_weights1 = np.dot(self.input.T,  (np.dot(2*(self.y - self.output) * sigmoid_derivative(self.output), self.weights2.T) * sigmoid_derivative(self.layer1)))

In [51]:
weights1 = np.array([[-1,1],[-2,2],[-3,3]])

In [56]:
x = np.array([1,1,1]).reshape(-1,1)
x

array([[1],
       [1],
       [1]])

In [57]:
weights1.dot(x)

ValueError: shapes (3,2) and (3,1) not aligned: 2 (dim 1) != 3 (dim 0)

In [16]:
width_list = [5,10,25,50,100]

In [128]:
net = NeuralNet(n_input_nodes=3, n_output_nodes=1, n_hidden_nodes=3)

In [129]:
net.weights[0][:,1:]

array([[-2, -3],
       [ 2,  3]])

In [130]:
x = np.array([1,1,1])

In [131]:
net.feedforward(x)

layer1 z=[-6  6] a=[1.         0.00247262 0.99752738]
layer2 z=[-3.99752738  3.99752738] a=[1.         0.01802994 0.98197006]


array([-2.43689523])

In [132]:
net.backprop(1)

[ 0.13598583 -0.10198937]


ValueError: shapes (2,) and (3,) not aligned: 2 (dim 0) != 3 (dim 0)

In [170]:
net.d_weights3

array([[28.78521271],
       [ 0.51899552],
       [28.26621719]])

In [141]:
a = np.array([1,2,3]).reshape(-1,1)
a, a.shape

(array([[1],
        [2],
        [3]]), (3, 1))

In [147]:
b = np.array([2])
b.shape

(1,)

In [152]:
np.dot(a,b)

array([2, 4, 6])

In [91]:
hw = (70+85+82+82+88+80) / 6
final = 80
report = 40

In [92]:
.5*hw + .2*final + .3*report

68.58333333333334

In [89]:
(hw*.5+final*.2)/.7

80.83333333333334

In [90]:
80*.7 + 50*.3

71.0